# Automated Brain Extraction
In this code we use recon-all funtion for brain extraction.The recon-all command from FreeSurfer software is used to process MRI scans, performing automated brain extraction and cortical reconstruction.


-----------------------------------------------------------
Script written by Sepideh Tabrik & Mehdi Behroozi
<br>
Biopsychology, 
<br>
Ruhr-University Bochum, Bochum, Germany
<br>
(2022.03)

-----------------------------------------------------------

In [ ]:
%%bash
# Define SUBJECTS_DIR explicitly
SUBJECTS_DIR="/mnt/d/Data/Human/ObjectCategorization/analysis2"

# Ensure Freesurfer directory exists
mkdir -p "$SUBJECTS_DIR/BrainExtraction"

# List relevant T1w files from session 01, extract subject IDs, and process in parallel
ls "$SUBJECTS_DIR"/sub*/ses-01/anat/*T1w.nii.gz -1 | awk -F'/' '{print $8}' | \
parallel --jobs 4 recon-all -s "$SUBJECS_DIR/BrainExtraction/anat_ses-01_{}" \
-i "$SUBJECTS_DIR/{}/ses-01/anat/{}*_ses-01_T1w.nii.gz" -all -qcache -3T

# List relevant T1w filesfrom sesion 02, extract subject IDs, and process in parallel
ls "$SUBJECTS_DIR"/sub*/ses-02/anat/*T1w.nii.gz -1 | awk -F'/' '{print $8}' | \
parallel --jobs 4 recon-all -s "$SUBJECTS_DIR/BrainExtraction/anat_ses-02_{}" \
-i "$SUBJECTS_DIR/{}/ses-02/anat/{}*_ses-02_T1w.nii.gz" -all -qcache -3T


#move results to the Freesurfer folder
mv "$SUBJECTS_DIR"/anat_ses-0* "$SUBJECTS_DIR/BrainExtraction/"

# Conver Brain Masks to nifti format
In addition to using recon-all from FreeSurfer for brain reconstruction, we also applied BET from the FSL package to enhance brain extraction, ensuring greater clarity and accuracy in the results.

In [ ]:
import glob
import os
from pathlib import Path

# Define data path
data_path = Path("/mnt/d/Data/Human/ObjectCategorization/analysis2")

# Find anatomical directories
anat_dirs = list(data_path.glob("BrainExtraction/anat*/mri"))

# Process each anatomy directory
for curr_anat in anat_dirs:
    print(f"Change Directory to >> {curr_anat}")
    

    # Ensure the directory exists before changing into it
    if not curr_anat.exists():
        print(f"Skipping: Directory {curr_anat} does not exist!")
        continue
    
    os.chdir(curr_anat)
    
    # Convert brain.mgz to brain.nii.gz
    print("\t1) Converting brain.mgz to brain.nii.gz.")
    os.system(f"mri_convert --reslice_like orig/001.mgz brain.mgz brain.nii.gz")

    # Run BET function from FSL package to get a clear brain mask
    print("\t2) Running BET function from FSL package...")
    os.system("bet brain brain_brain -R -f 0.3 -g 0 -m")

    # Define output path dynamically
    subject = curr_anat.parts[-2].split("_")[-1]
    session = curr_anat.parts[-2].split("_")[-2]
    out_put = data_path / subject / session / "anat" / "mask.nii.gz"

    print(f"\t3) Copying brain mask to {out_put.parent}")
    os.system(f"cp brain_brain_mask.nii.gz {out_put}")

    # Apply brain mask on the original image
    print("\t4) Applying brain mask on original image")
    t1w_image = list(out_put.parent.glob("*T1w.nii.gz"))
    out_put_brain = data_path / subject / session / "anat" / "brain.nii.gz"

    if t1w_image:
        os.system(f"fslmaths {t1w_image[0]} -mul {out_put} {out_put_brain}")
    else:
        print(f"Skipping: No T1w.nii.gz file found in {out_put.parent}")

    # Deleting the nifti files from Freesurfer folders
    os.system('rm -r *nii.gz')
    print("\n")
